<a href="https://colab.research.google.com/github/DarShivank/QuestionGenerator/blob/main/Question_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=9ababe17a9c942977e6f2b919dc75b486e2082e5b0a48a4976262dc0bb51ca18
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, BertTokenizer, BertModel, AutoTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import spacy
from transformers import BertTokenizer, BertModel
from warnings import filterwarnings as filt

filt('ignore')

In [4]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained("bert-base-uncased")
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
nlp = spacy.load("en_core_web_sm")

In [5]:
def get_question(sentence, answer):

  mdl = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
  tknizer = AutoTokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')

  text = "context: {} answer: {}".format(sentence,answer)
  max_len = 256
  encoding = tknizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = mdl.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=300)


  dec = [tknizer.decode(ids,skip_special_tokens=True) for ids in outs]


  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question

In [6]:
def get_embedding(doc):

  bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  bert_model = BertModel.from_pretrained("bert-base-uncased")

  # txt = '[CLS] ' + doc + ' [SEP]'
  tokens = bert_tokenizer.tokenize(txt)
  token_idx = bert_tokenizer.convert_tokens_to_ids(tokens)
  segment_ids = [1] * len(tokens)

  torch_token = torch.tensor([token_idx])
  torch_segment = torch.tensor([segment_ids])

  return bert_model(torch_token, torch_segment)[-1].detach().numpy()

def get_pos(context):
  doc = nlp(context)
  docs = [d.pos_ for d in doc]
  return docs, context.split()

def get_sent(context):
  doc = nlp(context)
  return list(doc.sents)

def get_vector(doc):
  stop_words = "english"
  n_gram_range = (1,1)
  df = CountVectorizer(ngram_range = n_gram_range, stop_words = stop_words).fit([doc])
  return df.get_feature_names_out()


def get_key_words(context, module_type = 't'):
  keywords = []
  top_n = 5
  for txt in get_sent(context):
    keywd = get_vector(str(txt))
    print(f'vectors : {keywd}')
    if module_type == 't':
      doc_embedding = get_embedding(str(txt))
      keywd_embedding = get_embedding(' '.join(keywd))
    else:
      doc_embedding = model.encode([str(txt)])
      keywd_embedding = model.encode(keywd)

    distances = cosine_similarity(doc_embedding, keywd_embedding)
    print(distances)
    keywords += [(keywd[index], str(txt)) for index in distances.argsort()[0][-top_n:]]

  return keywords

In [7]:
txt = 'Mauricio Pochettino open to leaving Paris St-Germain if Man Utd make approach'
get_key_words(txt)

vectors : ['approach' 'germain' 'leaving' 'make' 'man' 'mauricio' 'open' 'paris'
 'pochettino' 'st' 'utd']
[[1.]]


[('approach',
  'Mauricio Pochettino open to leaving Paris St-Germain if Man Utd make approach')]

In [8]:
txt = 'Mauricio Pochettino open to leaving Paris St-Germain if Man Utd make approach'
for ans, context in get_key_words(txt, 'st'):
  print('=======================================')
  print()
  print(get_question(context, ans))
  print()


vectors : ['approach' 'germain' 'leaving' 'make' 'man' 'mauricio' 'open' 'paris'
 'pochettino' 'st' 'utd']
[[0.38606033 0.475764   0.36602828 0.28078708 0.31993672 0.41910356
  0.47562957 0.58858883 0.40264374 0.30833417 0.33931935]]



You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Who is open to leaving Paris St-Germain if Man Utd make an approach?


Who is open to leaving Paris St-Germain if Man Utd make an approach?


Is Mauricio Pochettino open to leaving Paris St-Germain?


Pochettino is open to leaving Paris St-Germain if Man Utd make an approach?


What city is Mauricio Pochettino open to leaving if Man Utd make an approach?



In [9]:
import json

In [10]:
txt = 'Mauricio Pochettino open to leaving Paris St-Germain if Man Utd make approach'
generated_questions = []
common_answer = txt

for ans, context in get_key_words(txt, 'st'):
  question = get_question(context, ans)
  generated_questions.append(question)

# Create a dictionary with the questions and common answer
output_data = {
  'questions': generated_questions,
  'answer': [common_answer]
}

# Save the output data in a JSON file
output_filename = 'generated_questions2.json'

with open(output_filename, 'w') as json_file:
  json.dump(output_data, json_file, indent=4, ensure_ascii=False)

print(f'Successfully saved the generated questions in "{output_filename}" file.')

vectors : ['approach' 'germain' 'leaving' 'make' 'man' 'mauricio' 'open' 'paris'
 'pochettino' 'st' 'utd']
[[0.38606033 0.475764   0.36602828 0.28078708 0.31993672 0.41910356
  0.47562957 0.58858883 0.40264374 0.30833417 0.33931935]]
Successfully saved the generated questions in "generated_questions2.json" file.


In [11]:
paragraph = 'The Magnus EX can travel 80-100 km in a single charge. It also has a removable battery pack.The Magnus EX comes in 5 iconic colors- Metallic Red, Galactic Grey, Graphite Black, Glacial White and Ocean Blue'

sentences = get_sent(paragraph)
generated_questions = []
output_data = []

for sentence in sentences:
  sentence_text = str(sentence)
  sentence_questions = []

  for ans, context in get_key_words(sentence_text, 'st'):
    question = get_question(context, ans)
    sentence_questions.append(question)

  output_data.append({
    'sentence': sentence_text,
    'questions': sentence_questions,
    'answer': [sentence_text]
  })

# Save the output data in a JSON file
output_filename = 'generated_questions_para.json'

with open(output_filename, 'w') as json_file:
  json.dump(output_data, json_file, indent=4, ensure_ascii=False)

print(f'Successfully saved the generated questions in "{output_filename}" file.')

vectors : ['100' '80' 'charge' 'ex' 'km' 'magnus' 'single' 'travel']
[[0.44629598 0.48911572 0.24779609 0.16019392 0.24725953 0.2619614
  0.27319837 0.2890815 ]]
vectors : ['battery' 'pack' 'removable']
[[0.41366908 0.24487586 0.333483  ]]
vectors : ['black' 'blue' 'colors' 'comes' 'ex' 'galactic' 'glacial' 'graphite'
 'grey' 'iconic' 'magnus' 'metallic' 'ocean' 'red' 'white']
[[ 0.25818354  0.17140442  0.12329499 -0.01067966 -0.05660723  0.20775238
   0.24818927  0.23345815  0.12097265  0.14493924  0.14598545  0.2608771
   0.14972126  0.20610777  0.11726391]]
Successfully saved the generated questions in "generated_questions_para.json" file.
